# **Transfomer Architecture**


<img src="assets/architecture.png" alt="Screenshot" width="600" />


# Workflow
- input embeddings
- Positonal encoding
- Mutlihead attention(self attention)
- add and norm(layer normalization)
- feed forward network
- Residual network
- Encoder
- Decoder
- Building a Transformer
- testing
- tokenizer
- loading dataset
- validation loop
- training loop
- conclusion

# Importing libs

In [13]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
import math
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
from typing import Any
from tqdm import tqdm
import warnings

# Input Embedding